In [ ]:
import pandas as pd

In [ ]:
TRAINING_DATA_DIR = "labeled_data"
data = []
for dir in os.listdir(TRAINING_DATA_DIR):
    prompt = open(f'{TRAINING_DATA_DIR}/{dir}/prompt.md','r').read().strip()
    code = open(f'{TRAINING_DATA_DIR}/{dir}/code.md','r').read()
    data.append({'prompt':prompt, 'code':code})

